In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, TimeDistributed, SimpleRNN, GRU, LSTM
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dataset_path = '/content/drive/MyDrive/Datasets/joon_del/'
data_class_0 = os.path.join(dataset_path,'0')
data_class_1 = os.path.join(dataset_path,'1')

In [ ]:
%mkdir 'video_to_imgs'
%cd '/content/video_to_imgs/'
%mkdir '0'
%mkdir '1'

In [29]:
width = height = 70
batch_size = 8
epoches = 12
directory = '/content/video_to_imgs/'

In [5]:
def conver_to_dataset():

  frames = []
  labels = []
  
  # class 0
  for i in range(1, len(os.listdir(data_class_0))):
      video_cap = cv2.VideoCapture(f'{data_class_0}/0{i}.mp4')
      c = 0
      a_video_frames = []

      while True:
        success, frame = video_cap.read()
        if not success:
            break
        if c % 2 == 0:
          frame = cv2.resize(frame, dsize=(width, height)).astype('float32')
          a_video_frames.append(frame)

        c += 1

      # print('a_video_frames: ', len(a_video_frames))
      frames.append(a_video_frames)
      labels.append(0)


  # class 1
  for i in range(1, len(os.listdir(data_class_1))):
      video_cap = cv2.VideoCapture(f'{data_class_1}/0{i}.mp4')
      c = 0
      a_video_frames = []

      while True:
        success, frame = video_cap.read()
        if not success:
            break

        if c % 2 == 0:
          frame = cv2.resize(frame, dsize=(width, height)).astype('float32')
          a_video_frames.append(frame)

        c += 1
    
      # print('a_video_frames: ', len(a_video_frames))
      frames.append(a_video_frames)
      labels.append(1)

  labels = np.array(labels)

  return frames, labels

In [6]:
X_data, Y_data = conver_to_dataset()

In [7]:
len(X_data), len(X_data[0])

(68, 30)

In [ ]:
maximumـnumـofـframes = max(len(i) for i in X_data)
print('maximumـnumـofـframes: ', maximumـnumـofـframes)
for c, i in enumerate(X_data):
  if len(i) < maximumـnumـofـframes:
    print('len(i): ', len(i))
    for _ in range(maximumـnumـofـframes - len(i)):
      i.append(np.zeros((width, height, 3)))

  X_data[c] = np.array(X_data[c])

X_data = np.array(X_data)

In [9]:
len(X_data), len(X_data[0])

(68, 157)

In [10]:
X_data.shape, Y_data.shape

((68, 157, 70, 70, 3), (68,))

In [11]:
Y_data = Y_data[..., np.newaxis]

In [12]:
X_data.shape, Y_data.shape

((68, 157, 70, 70, 3), (68, 1))

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

In [30]:
model = tf.keras.models.Sequential([ 
                                   # CNN
                                   TimeDistributed(Conv2D(64, (3, 3), activation="relu", input_shape=(None, width, height, 3))), 
                                   TimeDistributed(MaxPooling2D(pool_size=(2, 2))), 
                                   TimeDistributed(Conv2D(16, (3, 3), activation="relu")), 
                                   TimeDistributed(Conv2D(4, (3, 3), activation="relu")), 
                                   TimeDistributed(MaxPooling2D(pool_size=(2, 2))), 
                                   TimeDistributed(Flatten()), 
                                   # RNN 
                                   SimpleRNN(40), 
                                   Dense(2, activation="softmax") ])

In [31]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

In [32]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = epoches)

Epoch 1/12


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7/7 [==============================] - 7s 673ms/step - loss: 0.6889 - accuracy: 0.4815
Epoch 2/12
7/7 [==============================] - 5s 674ms/step - loss: 0.6906 - accuracy: 0.5185
Epoch 3/12
7/7 [==============================] - 5s 663ms/step - loss: 0.6780 - accuracy: 0.5370
Epoch 4/12
7/7 [==============================] - 5s 673ms/step - loss: 0.6703 - accuracy: 0.6296
Epoch 5/12
7/7 [==============================] - 5s 651ms/step - loss: 0.6537 - accuracy: 0.5926
Epoch 6/12
7/7 [==============================] - 5s 669ms/step - loss: 0.6491 - accuracy: 0.6296
Epoch 7/12
7/7 [==============================] - 5s 681ms/step - loss: 0.6815 - accuracy: 0.5741
Epoch 8/12
7/7 [==============================] - 5s 652ms/step - loss: 0.6661 - accuracy: 0.6481
Epoch 9/12
7/7 [==============================] - 5s 674ms/step - loss: 0.6200 - accuracy: 0.6667
Epoch 10/12
7/7 [==============================] - 5s 668ms/step - loss: 0.6164 - accuracy: 0.7037
Epoch 11/12
7/7 [=============